In [2]:
import h5py
import numpy as np

In [4]:
# Define file names and dataset name
dirPath = '/Users/johnmarshall/Documents/Analysis/nVueData/SPRT/SPRT_m1_d6/'
input_file = 'concat_inscopix-1_2024-11-12-15-23-47_video_multiplexing_trig_0.h5'
dataset_name = '/1'
output_file_ch1 = '2024-11-12-15-23-47_channel1.h5'
output_file_ch2 = '2024-11-12-15-23-47_channel2.h5'

In [5]:
# Define a chunk size for processing (tweak this based on your available RAM)
chunk_size = 100  # number of frames to process per chunk

In [6]:
# Open the original file in read-only mode
with h5py.File(dirPath+input_file, 'r') as f_in:
    dset = f_in[dataset_name]
    total_frames, height, width = dset.shape
    # Determine the number of frames for each channel.
    # Even-numbered frames (0,2,4,...) go to channel 1.
    # Odd-numbered frames (1,3,5,...) go to channel 2.
    nframes_ch1 = total_frames // 2 + total_frames % 2
    nframes_ch2 = total_frames // 2
    # Create output files and datasets for each channel.
    with h5py.File(dirPath+output_file_ch1, 'w') as f_ch1, h5py.File(dirPath+output_file_ch2, 'w') as f_ch2:
        # Create datasets with the appropriate shape and same dtype
        dset_ch1 = f_ch1.create_dataset('data', shape=(nframes_ch1, height, width), dtype=dset.dtype, chunks=True)
        dset_ch2 = f_ch2.create_dataset('data', shape=(nframes_ch2, height, width), dtype=dset.dtype, chunks=True)      
        # Initialize write indices for each channel
        idx_ch1 = 0
        idx_ch2 = 0
        # Process the dataset in chunks along the first dimension
        for start in range(0, total_frames, chunk_size):
            end = min(start + chunk_size, total_frames)
            # Read a chunk from the original dataset
            chunk = dset[start:end, :, :]            
            # Deinterleave the frames:
            # Channel 1 gets even-indexed frames within the chunk
            ch1_chunk = chunk[0:(end - start):2]
            # Channel 2 gets odd-indexed frames within the chunk
            ch2_chunk = chunk[1:(end - start):2]
            # Write the chunks to the corresponding datasets
            n_ch1 = ch1_chunk.shape[0]
            n_ch2 = ch2_chunk.shape[0]
            dset_ch1[idx_ch1:idx_ch1+n_ch1, :, :] = ch1_chunk
            dset_ch2[idx_ch2:idx_ch2+n_ch2, :, :] = ch2_chunk 
            # Update indices for next write position
            idx_ch1 += n_ch1
            idx_ch2 += n_ch2  
            print(f"Processed frames {start} to {end} (ch1: {n_ch1} frames, ch2: {n_ch2} frames)")
print("Deinterleaving complete. Two files have been created for channel 1 and channel 2.")

Processed frames 0 to 100 (ch1: 50 frames, ch2: 50 frames)
Processed frames 100 to 200 (ch1: 50 frames, ch2: 50 frames)
Processed frames 200 to 300 (ch1: 50 frames, ch2: 50 frames)
Processed frames 300 to 400 (ch1: 50 frames, ch2: 50 frames)
Processed frames 400 to 500 (ch1: 50 frames, ch2: 50 frames)
Processed frames 500 to 600 (ch1: 50 frames, ch2: 50 frames)
Processed frames 600 to 700 (ch1: 50 frames, ch2: 50 frames)
Processed frames 700 to 800 (ch1: 50 frames, ch2: 50 frames)
Processed frames 800 to 900 (ch1: 50 frames, ch2: 50 frames)
Processed frames 900 to 1000 (ch1: 50 frames, ch2: 50 frames)
Processed frames 1000 to 1100 (ch1: 50 frames, ch2: 50 frames)
Processed frames 1100 to 1200 (ch1: 50 frames, ch2: 50 frames)
Processed frames 1200 to 1300 (ch1: 50 frames, ch2: 50 frames)
Processed frames 1300 to 1400 (ch1: 50 frames, ch2: 50 frames)
Processed frames 1400 to 1500 (ch1: 50 frames, ch2: 50 frames)
Processed frames 1500 to 1600 (ch1: 50 frames, ch2: 50 frames)
Processed fra

OSError: [Errno 28] Write through page buffer failed (file write failed: time = Mon Mar 10 16:20:57 2025
, filename = '/Users/johnmarshall/Documents/Analysis/nVueData/SPRT/SPRT_m1_d6/2024-11-12-15-23-47_channel1.h5', file descriptor = 75, errno = 28, error message = 'No space left on device', buf = 0x1781c0008, total write size = 202000, bytes this sub-write = 202000, offset = 5308015320)